# Importing and installing

In [ ]:
import torch

In [ ]:
import pandas as pd
import numpy as np
import os
import gc

In [ ]:
torch.__version__

'2.0.1+cu118'

In [ ]:
    # restart the session after running the cell
!pip install git+https://github.com/m-bain/whisperx.git

In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00


In [ ]:
from pytube import YouTube

In [ ]:
%cd /content/drive/MyDrive/datasets/speech_to_text

/content/drive/MyDrive/datasets/speech_to_text


In [ ]:
medium_df = pd.read_csv("medium_articles.csv")

In [ ]:
medium_df.head()

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


In [ ]:
print(medium_df['text'][0])

Photo by Josh Riemer on Unsplash

Merry Christmas and Happy Holidays, everyone!

We just wanted everyone to know how much we appreciate everyone and how thankful we are for all our readers and writers here. We wouldn’t be anywhere without you, so thank you all for bringing informative, vulnerable, and important pieces that destigmatize mental illness and mental health.

Without further ado, here are ten of our top stories from last week, all of which were curated:

“Just as the capacity to love and inspire is universal so is the capacity to hate and discourage. Irrespective of gender, race, age or religion none of us are exempt from aggressive proclivities. Those who are narcissistically disordered, and accordingly repress deep seated feelings of inferiority with inflated delusions of grandeur and superiority, are more prone to aggression and violence. They infiltrate our interactions in myriad environments from home, work, school and the cyber world. Hence, bullying does not happen in

In [ ]:
def time_concat(x):
    yt = YouTube(x['url'])
    return yt.length

In [ ]:
def title_concat(x):
    yt = YouTube(x['url'])
    return yt.title

In [ ]:
# concatenating time (in seconds) to the datasets
for x in os.listdir()[2:]:
    df = pd.read_csv(x)
    df['duration_(seconds)'] = df.apply(time_concat, axis=1)
    df.to_csv(x, index=False)
    print(f"finished concatenating on {x}")


In [4]:
# os.listdir()

In [ ]:
# concatenating title of the videos to the datasets

for x in os.listdir()[2:-1]:
    df = pd.read_csv(x)
    df_new = df[df['duration_(seconds)'] <= 1200]
    df_new['title'] = df_new.apply(title_concat, axis=1)
    df_new.to_csv(f"{x.split('.')[0]}_preprocessed.csv", index=False)
    print(f"finished adding titles on {x}")
    print(f"saved as {x.split('.')[0]}_preprocessed.csv")

## Helper function

In [ ]:
from pytube.exceptions import AgeRestrictedError

def audio_to_text(url, batch_size):
    f_name = 'audio_file.mp4'
    yt = YouTube(url)

    try:
        yt.streams.filter(only_audio=True)
        stream = yt.streams.get_by_itag(139)
        stream.download('', f_name)
        audio = whisperx.load_audio(f_name)
        result = model.transcribe(f_name, batch_size=batch_size, language='en')
        res = [ i['text'] for i in result['segments'] ]
        os.remove(f"{f_name}")
        torch.cuda.empty_cache()
        return res

    except AgeRestrictedError:
        res = ""
        return res


In [5]:
# os.listdir()

In [ ]:
gc.collect()
torch.cuda.empty_cache()

# Concatenating transcriptions from videos

In [ ]:
import whisperx

device = "cuda"
bs = 16
compute_type = "float16"
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

# for x in os.listdir()[1:-2]:
# technology smartphone done
df_name = 'Educational_statistics_preprocessed.csv'
df = pd.read_csv(df_name)
print(df.shape[0])
df['text'] = ""
for i in range(df.shape[0]):
    text = audio_to_text(df['url'][i], batch_size=bs)
    try:
        df.at[i, 'text'] = text[0]
        print(f"addded text at {i} / {df.shape[0]}")
    except IndexError:
        df.at[i, 'text'] = ""
        print(f"added empty string at {i} / {df.shape[0]}")
# df_new = df[df['duration_(seconds)'] <= 1200]
# df['text'] = df.apply(audio_to_speech, axis=1)
# df_new.to_csv(f"{x.split('.')[0]}_preprocessed.csv", index=False)
print(f"finished adding text on {df_name}")
df.to_csv(df_name, index=False)
print(f"saved as {df_name}")
# print(f"saved as {x.split('.')[0]}_preprocessed.csv")

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:01<00:00, 15.6MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.8. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../../../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu118. Bad things might happen unless you revert torch to 1.x.
124
addded text at 0 / 124
addded text at 1 / 124
addded text at 2 / 124
addded text at 3 / 124
addded text at 4 / 124
addded text at 5 / 124
addded text at 6 / 124
addded text at 7 / 124
addded text at 8 / 124
addded text at 9 / 124
addded text at 10 / 124
addded text at 11 / 124
addded text at 12 / 124
addded text at 13 / 124
addded text at 14 / 124
addded text at 15 / 124
addded text at 16 / 124
addded text at 17 / 124
addded text at 18 / 124
addded text at 19 / 124
addded text at 20 / 124
addded text at 21 / 124
addded text at 22 / 124
addded text at 23 / 124
addded text at 24 / 124
addded text at 25 / 124
addded text at 26 / 124
addded text at 27 / 124
addded text at 28 / 124
addded text at 29 / 124
addded text at 30 / 124
addded text at 

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
os.remove('transcribe_youtube_video.mp4')

In [ ]:
# for i in result['segments']:
#     print(i['text'])

res = [ i['text'] for i in result['segments']]
print(*res)

 I hope you will stay away from these programs.  The presidential election of South Asia's country, Maldives, is being held. The election is scheduled for 9th September. The small parties are sitting in such a situation. The small parties are sitting in such a situation. The small parties are sitting in such a situation.  According to the state election commission, if the candidates do not win the election on 9th September, they will have to vote in the second round. The voting date for the second round is also known as 30th September. With the support of the people, the residents of Bangladesh are not able to participate. We will wait for the election from our position, but we will not participate in any work with this election.  A beautiful environment has been created around this development. Roadside clubs, festoons, palakas are being filled. The number of voters registered in Maldives is about 2,30,000. Discussions on how much wind will be needed in Karpal in this state of develop